## **DyNet Installation**

In [1]:
! pip install -U dynet

     |████████████████████████████████| 4.4 MB 13.9 MB/s 


## **Static Networks**

In [29]:
# we assume that we have the dynet module in your path.
import dynet as dy
print('DyNet Version : %s' % dy.__version__)

DyNet Version : 2.1


In [30]:
# create a parameter collection and add the parameters.
model = dy.ParameterCollection()
Weights = model.add_parameters((8,2))
Vector = model.add_parameters((1,8))
bias = model.add_parameters((8))

dy.renew_cg() # new computation graph. not strictly needed here, but good practice.

In [31]:
Vector.value()

array([[ 0.01672149,  0.73200035,  0.39528334, -0.05091256,  0.70088458,
         0.56406248,  0.11054033, -0.49224973]])

In [32]:
Weights.value()

array([[ 0.02170771, -0.75533062],
       [ 0.75009274, -0.39546937],
       [-0.12265885, -0.74240756],
       [-0.41452768, -0.62821496],
       [ 0.7363255 ,  0.38632619],
       [-0.66921926, -0.47995621],
       [-0.17384517, -0.30395383],
       [ 0.69719303, -0.18562281]])

In [33]:
#b[1:-1].value()
bias.value()

[-0.43197232484817505,
 0.2133452296257019,
 0.054824650287628174,
 0.030038952827453613,
 -0.31308531761169434,
 -0.4460044503211975,
 -0.346992164850235,
 -0.4860454201698303]

In [34]:
x = dy.vecInput(2) # an input vector of size 2. Also an expression.
estimate = dy.logistic(Vector*(dy.tanh((Weights*x)+bias)))

In [35]:
# we can now query our network
x.set([0,0])
estimate.value()

0.4759312570095062

In [36]:
# we want to be able to define a loss, so we need an input expression to work against.
y = dy.scalarInput(0) # this will hold the correct answer
loss = dy.binary_log_loss(estimate, y)

In [37]:
x.set([1,0])
y.set(0)
print(loss.value())

y.set(1)
print(loss.value())

0.7989975214004517
0.5974364280700684


### **Training**

In [38]:
trainer = dy.SimpleSGDTrainer(model)

In [39]:
x.set([1,0])
y.set(1)
loss_value = loss.value() # this performs a forward through the network.
print("the loss before step is:",loss_value)

# now do an optimization step
loss.backward()  # compute the gradients
trainer.update()

# see how it affected the loss:
loss_value = loss.value(recalculate=True) # recalculate=True means "don't use precomputed value"
print("the loss after step is:",loss_value)


the loss before step is: 0.5974364280700684
the loss after step is: 0.5264061689376831


In [40]:
def training_set_xor(num_rounds=2000):
    inputs = []
    outputs = []
    for round in range(num_rounds):
        for x1 in 0,1:
            for x2 in 0,1:
                output = 0 if x1==x2 else 1
                inputs.append((x1,x2))
                outputs.append(output)
    return inputs, outputs

inputs, outputs = training_set_xor()

In [41]:
total_loss = 0
seen_instances = 0
for input, output in zip(inputs, outputs):
    x.set(input)
    y.set(output)
    seen_instances += 1
    total_loss += loss.value()
    loss.backward()
    trainer.update()
    if (seen_instances > 1 and seen_instances % 100 == 0):
        print("average loss is:",total_loss / seen_instances)

average loss is: 0.7265273672342301
average loss is: 0.6818469610810279
average loss is: 0.6311899694800377
average loss is: 0.5744382533431053
average loss is: 0.514801154255867
average loss is: 0.45856580141931774
average loss is: 0.4097782119682857
average loss is: 0.36884966315235945
average loss is: 0.3347053258534935
average loss is: 0.3060608646105975
average loss is: 0.28180533300238575
average loss is: 0.26105752408737315
average loss is: 0.24313562335733038
average loss is: 0.22751362519238943
average loss is: 0.2137833709906166
average loss is: 0.20162521373975323
average loss is: 0.19078620544388233
average loss is: 0.1810640603534153
average loss is: 0.1722953453841374
average loss is: 0.16434673514438328
average loss is: 0.15710847434820607
average loss is: 0.15048944258977745
average loss is: 0.14441339623618304
average loss is: 0.13881607098766835
average loss is: 0.13364293185407297
average loss is: 0.12884740758633528
average loss is: 0.1243894949288073
average loss i

In [42]:
x.set([0,1])
print("0,1", estimate.value())

x.set([1,0])
print("1,0", estimate.value())

x.set([0,0])
print("0,0", estimate.value())

x.set([1,1])
print("1,1", estimate.value())

0,1 0.9973140358924866
1,0 0.9975988268852234
0,0 0.00048306540702469647
1,1 0.0026758129242807627


In [43]:
Weights.value()

array([[ 1.56691289, -2.66784978],
       [ 0.90280598, -2.03891754],
       [ 1.51222038, -2.80169749],
       [-0.98409206, -1.04656172],
       [ 3.22858667,  3.26019168],
       [-2.80845404,  1.66872609],
       [-0.25202587, -0.2965796 ],
       [ 3.12019157, -1.70047855]])

In [44]:
Vector.value()

array([[ 2.69440007,  1.68956578,  2.96427298, -1.29539144,  5.82483053,
         3.17031765,  0.73548168, -3.83105183]])

In [45]:
bias.value()

[-0.4088715612888336,
 -0.02462054044008255,
 -0.30127471685409546,
 -0.5590880513191223,
 -0.9468629360198975,
 -0.41303399205207825,
 0.5945218205451965,
 0.32148823142051697]